# 2024년 따릉이 이용건수 데이터 확인

In [13]:
import pandas as pd
import numpy as np
import os
import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)

### csv파일 합치기
#### 24년 공공자전거 이용정보(시간대별) 데이터 병합

- 서울특별시 공공자전거 이용정보(시간대별)_202401.csv
- 서울특별시 공공자전거 이용정보(시간대별)_202402.csv
- 서울특별시 공공자전거 이용정보(시간대별)_202403.csv
- 서울특별시 공공자전거 이용정보(시간대별)_202404.csv
- 서울특별시 공공자전거 이용정보(시간대별)_202405.csv
- 서울특별시 공공자전거 이용정보(시간대별)_202406.csv

병합할 파일은 하나의 폴더에 존재해야 함

In [ ]:
folder_path = 'C:/Users/GBOOK/Codeit/초급프로젝트/02_data/03_따릉이/24년'  # CSV 파일들이 저장된 폴더 경로
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

In [ ]:
# 빈 리스트 생성
dataframes = []

# 각 CSV 파일을 읽어서 리스트에 추가
for file in csv_files:
    df = pd.read_csv(file, encoding='cp949')  # 필요시 옵션 추가 (예: encoding='utf-8-sig')
    dataframes.append(df)

# 데이터프레임 병합
combined_df = pd.concat(dataframes, ignore_index=True)  # 인덱스 초기화


In [ ]:
combined_df.to_csv('combined_file.csv', index=False, encoding='cp949')

### 병합이 완료된 이후에는 병합된 csv파일 불러오기

In [2]:
start_time = time.time()
combined_df = pd.read_csv("combined_file.csv", usecols=['대여일자', '대여시간', '대여소번호', '대여소명', '이용건수'], encoding='cp949')
end_time = time.time()

print(f"시간:{end_time - start_time:.2f}초")

시간:53.09초


### 데이터 확인하기

In [3]:
combined_df.head()

,대여일자,대여시간,대여소번호,대여소명,이용건수
0,2024-01-01,0,1175,1175. 대한항공 인력개발센터,1
1,2024-01-01,0,669,669.청계한신휴플러스앞 삼거리,1
2,2024-01-01,0,5760,5760. 풍납사거리 성내유수지,1
3,2024-01-01,0,1153,"1153. 발산역 1번, 9번 인근 대여소",1
4,2024-01-01,0,1265,1265. 문정동 근린공원,1


In [4]:
combined_df.shape

(19369651, 5)

In [5]:
combined_df.isna().sum()

대여일자     0
대여시간     0
대여소번호    0
대여소명     0
이용건수     0
dtype: int64

In [6]:
combined_df.columns

Index(['대여일자', '대여시간', '대여소번호', '대여소명', '이용건수'], dtype='object')

In [7]:
# 이용건수는 해당 시간대에 성별과 나이대가 동일한 사람이 대여한 건수
# 즉 시간대별 이용건수를 합치면 그 시간대에 전체 이용건수 확인 가능
combined_df['이용건수'].unique()

array([ 1,  2,  3,  6,  7,  4,  5,  9,  8, 16, 10, 12, 15, 11, 13, 19, 14,
       18, 17, 24, 20, 22, 30, 39, 25, 21, 29, 23, 26, 28, 27, 31])

In [ ]:
# 필요없는 컬럼 삭제하기
cols = ['대여구분코드', '성별', '연령대코드', '운동량', '탄소량', '이동거리(M)', '이용시간(분)']

drop_df = combined_df.drop(columns=cols)

In [ ]:
drop_df['이용건수'].unique()

In [8]:
# csv합친 파일을 대여일자, 대여시간, 대여소번호, 대여소명 으로 그룹화 > 이용건수 총합 구하기
group = combined_df.groupby(['대여일자', '대여시간', '대여소번호', '대여소명'], as_index=False).agg({'이용건수':'sum'})

group.rename(columns={'이용건수' : '총이용건수'}, inplace=True)

In [10]:
# 따릉이 대여소 마스터 정보 불러오기
master_df = pd.read_csv("C:/Users/GBOOK/Codeit/초급프로젝트/02_data/03_따릉이/24년/공공자전거_대여소_정보_2024.csv", encoding='cp949')

In [11]:
# 마스터정보와 그룹화된 데이터 병합
merged_df = pd.merge(group, master_df, on='대여소번호', how='inner')

### 이용건수 계산

In [12]:
# 같은 대여일자와 대여소에서 하루 동안 빌린 총 이용건수 계산
daily_total = merged_df.groupby(['대여일자', '대여소번호', '대여소명', '자치구', '거치대수'], as_index=False).agg({'총이용건수': 'sum'})

# 열 이름 확인 및 필요시 조정
daily_total.rename(columns={'총이용건수': '하루총이용건수'}, inplace=True)

In [14]:
daily_total = daily_total[['대여일자', '대여소번호', '대여소명', '자치구', '하루총이용건수', '거치대수']]

In [15]:
# 강남구.서초구 데이터 추출
# 분석할려는 강남구 대여소가 서초구에 인접하여 서초구 데이터도 추출
gangnamgu_24 = daily_total[daily_total['자치구'].isin(['강남구', '서초구'])]

In [16]:
gangnamgu_24.head(3)

,대여일자,대여소번호,대여소명,자치구,하루총이용건수,거치대수
1392,2024-01-01,2201,2201. 본마을 정류소 앞,서초구,1,14
1393,2024-01-01,2202,2202.청계산입구역 1번출구,서초구,12,20
1394,2024-01-01,2203,2203. 청계산입구역 2번출구,서초구,7,10


In [17]:
# 서대문구 데이터 추출
seodaemungu_24 = daily_total[daily_total['자치구'].isin(['서대문구'])]

In [19]:
seodaemungu_24.head(3)

,대여일자,대여소번호,대여소명,자치구,하루총이용건수,거치대수
10,2024-01-01,115,115. 마스타 빌딩 앞,서대문구,19,15
11,2024-01-01,117,117. 홍은사거리,서대문구,34,23
17,2024-01-01,123,123. 문화촌 공원,서대문구,36,20


In [20]:
# 중구 데이터 추출
junggu_24 = daily_total[daily_total['자치구'].isin(['중구'])]

In [21]:
junggu_24.head(3)

,대여일자,대여소번호,대여소명,자치구,하루총이용건수,거치대수
162,2024-01-01,300,300. 정동사거리,중구,14,6
170,2024-01-01,310,310. 청계광장 옆,중구,7,8
173,2024-01-01,318,318. 광교사거리 남측,중구,4,10


### 월별로 대여소별 하루총이용건수 합

In [22]:
# 대여일자 데이터 타입을 데이트타임으로 변경
gangnamgu_24['대여일자'] = pd.to_datetime(gangnamgu_24['대여일자'])

# 월 단위 대여월 컬럼 생성
gangnamgu_24['대여월'] = gangnamgu_24['대여일자'].dt.to_period("M")

In [23]:
# 대여일자 type 확인
seodaemungu_24['대여일자'] = pd.to_datetime(seodaemungu_24['대여일자'])

# 월 단위 대여월 컬럼 생성
seodaemungu_24['대여월'] = seodaemungu_24['대여일자'].dt.to_period("M")

In [24]:
# 대여일자 type 확인
junggu_24['대여일자'] = pd.to_datetime(junggu_24['대여일자'])

# 월 단위 대여월 컬럼 생성
junggu_24['대여월'] = junggu_24['대여일자'].dt.to_period("M")

In [25]:
# 대여소별, 대여월별 하루총이용건수 합계 계산
gangnamgu_month = gangnamgu_24.groupby(['대여월', '대여소명', '대여소번호', '거치대수'])['하루총이용건수'].sum().reset_index()
gangnamgu_month.rename(columns={'하루총이용건수': '월별총이용건수'}, inplace=True)

# 대여월을 보기 좋게 변환
gangnamgu_month['대여월'] = gangnamgu_month['대여월'].dt.strftime('%Y-%m')

In [26]:
# 대여소별, 대여월별 하루총이용건수 합계 계산
seodaemungu_month = seodaemungu_24.groupby(['대여월', '대여소명', '대여소번호', '거치대수'])['하루총이용건수'].sum().reset_index()
seodaemungu_month.rename(columns={'하루총이용건수': '월별총이용건수'}, inplace=True)

# 대여월을 보기 좋게 변환
seodaemungu_month['대여월'] = seodaemungu_month['대여월'].dt.strftime('%Y-%m')

In [27]:
# 대여소별, 대여월별 하루총이용건수 합계 계산
junggu_month = junggu_24.groupby(['대여월', '대여소명', '대여소번호', '거치대수'])['하루총이용건수'].sum().reset_index()
junggu_month.rename(columns={'하루총이용건수': '월별총이용건수'}, inplace=True)

# 대여월을 보기 좋게 변환
junggu_month['대여월'] = junggu_month['대여월'].dt.strftime('%Y-%m')

### 월별 대여소의 평균 사용지수 생성
월별총이용건수/거치대수

In [28]:
# 중구 월별 사용지수
junggu_month['월별사용지수'] = junggu_month['월별총이용건수'] / junggu_month['거치대수']

junggu_month['월별사용지수'] = junggu_month['월별사용지수'].round(1)

In [33]:
junggu_month.head(2)

,대여월,대여소명,대여소번호,거치대수,월별총이용건수,월별사용지수
0,2024-01,300. 정동사거리,300,6,703,117.2
1,2024-01,3013.서울지방고용노동청 앞,3013,10,862,86.2


In [29]:
# 서대문구 월별 사용지수
seodaemungu_month['월별사용지수'] = seodaemungu_month['월별총이용건수'] / seodaemungu_month['거치대수']

seodaemungu_month['월별사용지수'] = seodaemungu_month['월별사용지수'].round(1)

In [32]:
seodaemungu_month.head(2)

,대여월,대여소명,대여소번호,거치대수,월별총이용건수,월별사용지수
0,2024-01,115. 마스타 빌딩 앞,115,15,524,34.9
1,2024-01,117. 홍은사거리,117,23,1214,52.8


In [30]:
# 강남구 월별 사용지수
gangnamgu_month['월별사용지수'] = gangnamgu_month['월별총이용건수'] / gangnamgu_month['거치대수']

gangnamgu_month['월별사용지수'] = gangnamgu_month['월별사용지수'].round(1)

In [34]:
gangnamgu_month.head(2)

,대여월,대여소명,대여소번호,거치대수,월별총이용건수,월별사용지수
0,2024-01,2201. 본마을 정류소 앞,2201,14,38,2.7
1,2024-01,2202.청계산입구역 1번출구,2202,20,903,45.2


### csv 파일 저장

In [ ]:
junggu_month.to_csv("2024_중구_월별사용지수.csv",  encoding='cp949')

In [ ]:
seodaemungu_month.to_csv("2024_서대문구_월별사용지수.csv",  encoding='cp949')

In [ ]:
gangnamgu_month.to_csv("2024_강남구_월별사용지수.csv",  encoding='cp949')